# Time Dependent Schrodinger Equation
    Marienne Faro - 4229592
    Tim Gebraad - 4247612

    AP3082D - Computational Physics - Project 3

This notebook contains the results for solving the time dependent Schrödinger equation in 1D and 2D for different types of potential landscapes. The methods used to solve these TDSE are Crank-Nicholson, which is basicly a dicretization of the differential equation. The other method is the Split Operator (SO) which calculates the quantum mechanical time evolution of a non-commuting Hamiltonian by a Lie, Suzuki, Trotter process, where the time evolution of the two parts of the Hamiltonian are rapidly alternated, resulting in an error that scales with the timestep squared.

(The units used can be derived from setting the electron mass to 1 and $\hbar=0$, which simplifies the SE) 

### 1D TDSE 

In [70]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.sparse.linalg
import scipy.sparse as sparse
from scipy.sparse import spdiags
from scipy.sparse import csr_matrix
import time

In [87]:
class TDSE_1D(object):
    def __init__(self):
        return
        
    def SetSpace(self, xmin, xmax, Xsteps):
        self.x = np.linspace(xmin,xmax,Xsteps)
        self.X = Xsteps
        self.dx = (xmax-xmin)/self.X
        
        self.k = np.fft.fftfreq(self.X, self.dx)
        return
                
    def SetPotential(self, V, param=None):
        self.V = np.zeros(self.X)
        if V=='Infinite Barrier':
            self.V[int(0.52*self.X):self.X-1] = 2**10
        elif V=='Tunneling Barrier':
            [w,h] = param
            self.V[int(0.52*self.X):int(0.52*self.X+w/self.dx)] = h
        elif V=='Infinite Well':
            a = param/2
            self.V[0:int(self.X/2-a/self.dx)]        = 2**10
            self.V[int(self.X/2+a/self.dx):self.X-1] = 2**10
        elif V=='Harmonic Oscillator':
            k0 = param[0]
            self.V = k0*self.x**2
        elif V=='Lattice Sine':
            [A, k0] = param
            self.V = A-A*np.cos(k0*self.x)
        elif V=='Lattice Delta':
            [a, b, h] = param
            N = int(self.X*self.dx/b)
            dn = int(self.X/N)
            for n in range(N):
                self.V[n*dn:n*dn+int(a/self.dx)] = h
        else:
            self.V = V
            
    def SetInitialWave(self, k0=1, sigma=0.05):
        self.Psi0_k = np.exp(-(self.k-k0)**2/(2*sigma))*(np.pi*sigma)**(-1/4)
        self.Psi0_x = np.fft.fftshift(np.fft.ifft(self.Psi0_k, norm="ortho"))
        return
        
    def SetupSimulation(self, dt, Tsteps):     
        self.t = np.linspace(0, Tsteps*dt, Tsteps)
        self.dt = dt
        self.T = Tsteps
        
        self.Psi_x = np.zeros((self.T, self.X), dtype = complex)
        self.Psi_k = np.zeros((self.T, self.X), dtype = complex)
        self.Psi_x[0,:] = self.Psi0_x
        self.Psi_k[0,:] = self.Psi0_k
        return
        
    def SimulateCrankNicholson(self, dt=0.05, Tsteps=10000):      
        self.SetupSimulation(dt, Tsteps)

        #Setup the Hamiltonian
        P = np.ones((3,self.X))
        P[1,:] *=-2
        H = -1/2*spdiags(P,[-1, 0, 1],self.X,self.X)+spdiags(self.V,0,self.X,self.X)
        
        #Setup the left and right hand side matrix multiplicator
        Hl = 1j/self.dt*scipy.sparse.identity(self.X)-1/2*H
        Hr = 1j/self.dt*scipy.sparse.identity(self.X)+1/2*H
        
        for t in range(self.T-1):
            self.Psi_x[t+1,:] = sparse.linalg.bicgstab(Hl, np.dot(Hr.toarray(),self.Psi_x[t,:]))[0]
        return        
        
    def SimulateSplitOperator(self, dt=0.01, Tsteps=10000):       
        self.SetupSimulation(dt, Tsteps)
        
        EvltPos = np.exp(-1j*self.dt*self.V) 
        EvltMom = np.exp(-1j*self.dt*abs(self.k)**2)
        
        for t in range(self.T-1):
            self.Psi_k[t+1,:] = np.fft.fft (EvltPos*self.Psi_x[t  ,:], norm="ortho")
            self.Psi_x[t+1,:] = np.fft.ifft(EvltMom*self.Psi_k[t+1,:], norm="ortho")
        return
               
    def AnimateSetup(self):
        self.fig, (self.axPos, self.axMom) = plt.subplots(2,1, facecolor='white')
        self.axPos.fill(np.concatenate(([self.x[0]],self.x,[self.x[self.X-1]])), np.concatenate(([-100],self.V,[-100])), 'k', alpha=0.8)
        self.linePsi_x, = self.axPos.plot(self.x, abs(self.Psi_x[0,:])**2, 'r')
        self.linePsi_k, = self.axMom.plot(self.k, abs(self.Psi_k[0,:])**2, 'y')
        self.axPos.set_ylim([np.min(self.V), 25])
        self.axPos.set_xlim([-20, 20])
        self.axPos.set_xlabel('$x$')
        self.axPos.set_ylabel(r'$|\psi(x)| , V(x)$')
        self.axMom.set_xlabel('$k$')
        self.axMom.set_ylabel(r'$|\psi(k)|$')
        self.axPos.legend(('$|\psi(x)|$', '$V(x)$'))
        self.axMom.legend(('$|\psi(k)|$',))
        self.text = plt.text(-5, 2.7, "") 
        return
        
    def AnimationStep(self, i):
        self.linePsi_x.set_ydata(abs((self.Psi_x[i%self.T,:]))**2)
        self.linePsi_k.set_ydata(abs((self.Psi_k[i%self.T,:]))**2)
        self.text.set_text("t = %.2f" % self.t[i%self.T])
        return (self.linePsi_x, self.linePsi_k, self.text)

First set up the 1D TDSE object, with the appropriate space and initial Gaussian Wave

In [88]:
MyTDSE = TDSE_1D()
MyTDSE.SetSpace(xmin=-100, xmax=100, Xsteps=2001)
MyTDSE.SetInitialWave(k0=4, sigma=0.05)

First look at an inifinite barriere, where the Gaussian wave reflects for Crank-Nicholson and the Split operator method

In [73]:
MyTDSE.SetPotential('Infinite Barrier')
MyTDSE.SimulateCrankNicholson(Tsteps=3000)
MyTDSE.AnimateSetup()
ani = animation.FuncAnimation(MyTDSE.fig, MyTDSE.AnimationStep, interval=10, blit=True)
plt.show()

In [74]:
MyTDSE.SimulateSplitOperator(Tsteps=3000)
MyTDSE.AnimateSetup()
ani = animation.FuncAnimation(MyTDSE.fig, MyTDSE.AnimationStep, interval=10, blit=True)
plt.show()

Now for a tunneling barrier, classicly a wave cannot pass this barrier if it's to heigh ($V(x)>\frac{1}{2}p^2$), but quantum mechanically there is a possibility of tunneling (transmitting). This possibility can be looked at by varying the initial velocity of the wave or the potential barrier. From now on the Split Operator method is used, because it turns out to be computationally faster.

In [9]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, facecolor='white')
ax = (ax1, ax2, ax3)

#Set ranges for the width (A), wavenumbers (K) and height(V) that are analyzed
A = [0.1, 0.5, 1.0]
K = np.arange(0.0, 4, 0.2)
V = np.array([0.1, 3, 7, 10])
c= ['r', 'g', 'b', 'm']
T = np.zeros((len(K), len(V)))

#Perform the simulation and obtain the tranmission for all combinations
for n, a in enumerate(A):
    for i, k in enumerate(K):
        MyTDSE.SetInitialWave(k0=k, sigma=0.05)
        for j, v in enumerate(V):
            MyTDSE.SetPotential('Tunneling Barrier', [a, v])
            MyTDSE.SimulateSplitOperator(Tsteps=3000)
            T[i,j] = np.sum(abs(MyTDSE.Psi_x[-1, int(0.52*MyTDSE.X):-1])**2)/np.sum(abs(MyTDSE.Psi_x[-1])**2)

    for j, v in enumerate(V):
        ax[n].plot(K, T[:, j], color=c[j], marker='*', markeredgecolor=c[j])
        
    #Plot tweaking
    ax[n].set_xlabel('$k_0$')
    ax[n].set_ylabel('$Transmission$')
    ax[n].set_title('Width ='+ str(a))
    if n==0:
        ax[n].legend(('$V_{max} = 0.1$', '$V_{max} = 3$', '$V_{max} = 7$', '$V_{max} = 10$'))
        
    #Plot the classical boundary and the theoretical (quantum) expectation value
    for j, v in enumerate(V):
        ax[n].plot(np.ones(2)*np.sqrt(V[j]), [0,1], lw=5, color=c[j], alpha=0.3)
        k1 = np.lib.scimath.sqrt(K**2-v*2)
        Tth = 4*K*k1*np.exp(-1j*a*(K-k1))/((K+k1)**2-np.exp(2j*a*k1)*(K-k1)**2)
        ax[n].plot(K, abs(Tth)**2, color=c[j], ls=':')
plt.show()

<img src="Results/Transmission.png">

The tranmission shows the expected shape when looking at the theoretical expectation (dotted), but it does differ quite a lot from it (we expect this to be a mistake with a constant). It is also seen that the tranmission indeed is non-zero in the quantum mechanical case, whereas this is classically (vertical lines) not possible.

Next, two confining potentials are implemented: the infinite potential well and the harmonic oscillator.

In [76]:
#Set up the appropriate energy for level n and well width a and give this as input to the TDSE
n = 1
a = 5
MyTDSE.SetInitialWave(k0=n*np.pi/a, sigma=0.001)
MyTDSE.SetPotential('Infinite Well', a)
MyTDSE.SimulateSplitOperator(Tsteps=10000)
MyTDSE.AnimateSetup()
ani = animation.FuncAnimation(MyTDSE.fig, MyTDSE.AnimationStep, interval=10, blit=True)
plt.show()

In [77]:
#Set up the appropriate energy level n and oscillator wavenumer k1 and give this as input to the TDSE
n = 1
k1 = 1


MyTDSE.SetInitialWave(k0=k1*np.sqrt(n+0.5), sigma=0.001)
MyTDSE.SetPotential('Harmonic Oscillator', [k1])
MyTDSE.SimulateSplitOperator(Tsteps=10000)
MyTDSE.AnimateSetup()
ani = animation.FuncAnimation(MyTDSE.fig, MyTDSE.AnimationStep, interval=10, blit=True)
plt.show()

We have runned simulations where we wanted to find the quantized levels of both the infinite well potential and the harmonic oscillator, by inserting the right energy via k. This was however unsuccesfull, maybe due to non-relaxation of the system or maybe due to other reasons.

Lastly, two periodic potentials are looked at. Firstly a simple sine potential and secondly a potential with delta peaks. Tweaking the parameters of the latter can result in a potential that resembles the one in a crystallic structure, so for example the behavoir of an electron within a crystal lattice can be studied (and compared to the free electron).

In [78]:
MyTDSE.SetInitialWave(k0=2, sigma=0.05)

In [79]:
MyTDSE.SetPotential('Lattice Sine', [1, 1])
MyTDSE.SimulateSplitOperator(Tsteps=5000)
MyTDSE.AnimateSetup()
ani = animation.FuncAnimation(MyTDSE.fig, MyTDSE.AnimationStep, interval=10, blit=True)
plt.show()

In [80]:
MyTDSE.SetPotential('Lattice Delta', [0.1, 2, -2])
MyTDSE.SimulateSplitOperator(Tsteps=5000)
MyTDSE.AnimateSetup()
ani = animation.FuncAnimation(MyTDSE.fig, MyTDSE.AnimationStep, interval=10, blit=True)
plt.show()

### TDSE 2D

And now for 2D. We tried to implement this for both the Crank-Nicholson and Split Operator method, but for CN we did not get the wave moving, so the experiments have been run with SO.
To create a plane wave we have used three boundary conditions:
- At the bottom (our case y=-4) the wave is forces to be:
    - $ \psi (x) = e^{i(kx-\omega t)}$
    - $ \frac{d\psi (x)}{dx} = ike^{i(kx-\omega t)}$
- And additionally, in order to avoid periodic boundary effects of the Fourier Transform at the top:
    - $ \psi (x) = 0$

In [81]:
class TDSE_2D(object):
    def __init__(self):
        return
        
    def SetSpace(self, xmin, xmax, Xsteps, ymin, ymax, Ysteps):
        self.x = np.linspace(xmin, xmax, Xsteps)
        self.X = Xsteps
        self.dx= (xmax-xmin)/self.X
        
        self.y = np.linspace(ymin, ymax, Ysteps)
        self.Y = Ysteps
        self.dy= (ymax-ymin)/self.Y
        
        self.kx = np.fft.fftfreq(self.X, self.dx)
        self.ky = np.fft.fftfreq(self.Y, self.dy)
        
        xx, yy = np.meshgrid(self.kx, self.ky)
        self.k = np.sqrt(xx**2 + yy**2)
        return
                
    def SetPotential(self, V):
        self.V = np.zeros((self.X, self.Y), dtype=complex)
        if V=='Empty':
            return
        if V=='Single Slit':
            self.V[int(0.15*self.X):int(0.16*self.X), int(0.00*self.Y):int(0.45*self.Y)] = 2**10
            self.V[int(0.15*self.X):int(0.16*self.X), int(0.55*self.Y):int(1.00*self.Y)] = 2**10
        elif V=='Double Slit':
            self.V[int(0.15*self.X):int(0.16*self.X), int(0.00*self.Y):int(0.40*self.Y)] = 2**10
            self.V[int(0.15*self.X):int(0.16*self.X), int(0.45*self.Y):int(0.55*self.Y)] = 2**10
            self.V[int(0.15*self.X):int(0.16*self.X), int(0.60*self.Y):int(1.00*self.Y)] = 2**10
        else:
            self.V = V
        
    def SetupSimulation(self, dt, Tsteps):
        self.t = np.linspace(0, (Tsteps-1)*dt, Tsteps)
        self.dt = dt
        self.T = Tsteps
        
        self.Psi_x = np.zeros((self.T, self.X, self.Y), dtype = complex)
        self.Psi_k = np.zeros((self.T, self.X, self.Y), dtype = complex)
        return
        
    def SimulateCrankNicolson(self, dt=0.01, Tsteps=1001):
        self.SetupSimulation(dt, Tsteps)
        
        #Set up the Hamiltonian
        P = np.ones((3,self.X))
        P[1,:] *=-2
        H = scipy.sparse.identity(self.X*self.Y)
        P = -np.ones(self.X*self.Y)
        for i in range (self.X-1):
            P[self.X*(i+1)] = 0
        H+= spdiags([np.concatenate((P[1:len(P)], [0])),P], [-1, 1], self.X*self.Y, self.X*self.Y)
        H+= spdiags(np.ones((2,self.X*self.Y)), [-self.X, self.X], self.X*self.Y, self.X*self.Y)
        H+= spdiags(np.reshape(self.V, (self.X*self.Y)),0, self.X*self.Y,self.X*self.Y)
        
        Hr = 2j/self.dt*scipy.sparse.identity(self.X*self.Y)-1/2*H
        Hl = 2j/self.dt*scipy.sparse.identity(self.X*self.Y)+1/2*H
        
        k0=1
        self.Psi_x[0] = np.zeros((self.X, self.Y), dtype=complex) 
        
        for t in range(self.T-1):
            print('Simulated timestep', self.t[t])
            
            #Use boundary conditions
            self.Psi_x[t, 0] = np.ones(self.Y)*np.exp(-1j*self.t[t]*np.pi)
            self.Psi_x[t, 1] = self.Psi_x[t, 0, :] + self.dx*np.ones(self.Y)*np.exp(-1j*self.t[t]*np.pi)*1j*k0
            self.Psi_x[t,-1] = 0
            
            #Calculate the next step
            Psi = np.reshape(self.Psi_x[t,:,:], (self.X*self.Y))
            Psi = sparse.linalg.bicgstab(Hl, Hr.dot(Psi))[0]
            self.Psi_x[t+1,:,:] = np.reshape(Psi, (self.X, self.Y)) 

        return
        
    def SimulateSplitOperator(self, dt=0.01, Tsteps=1001):       
        self.SetupSimulation(dt, Tsteps)

        EvltPos = np.exp(-1j*self.dt*self.V)
        EvltMom = np.exp(-1j*self.dt*abs(self.k)**2).transpose()
        
        k0 = 1
        self.Psi_x[0] = np.zeros((self.X, self.Y), dtype=complex)    
        
        for t in range(self.T-1):
            #Use boundary conditions
            self.Psi_x[t, 0] = np.ones(self.Y)*np.exp(-1j*self.t[t]*np.pi)
            self.Psi_x[t, 1] = self.Psi_x[t, 0, :] + self.dx*np.ones(self.Y)*np.exp(-1j*self.t[t]*np.pi)*1j*k0
            self.Psi_x[t,-1] = 0
            
            #Calculate the next step
            self.Psi_k[t+1] = np.fft.fft2 (EvltPos*self.Psi_x[t  ], norm="ortho")
            self.Psi_x[t+1] = np.fft.ifft2(EvltMom*self.Psi_k[t+1], norm="ortho")
        return
               
    def AnimateSetup(self, interval=10):
        self.fig, (self.ax1D, self.ax2D) = plt.subplots(1,2)
        self.ax2D.pcolormesh(self.x,self.y,np.real(self.V),cmap='Greys')
        self.quadPsi_x = self.ax2D.pcolormesh(self.x, self.y, abs(self.Psi_x[0])**2, shading='gouraud', alpha=0.1, cmap='plasma')
        
        self.linePsi_x, = self.ax1D.plot(self.y, abs(self.Psi_x[0,:,0])**2, 'y')        
        
        self.ax2D.set_xlabel('$x$')
        self.ax2D.set_ylabel('$y$')
        self.ax2D.set_title('Position')
        self.ax1D.set_xlabel('$x$')
        self.ax1D.set_ylabel(r'$|\psi(x)|$')
        self.ax1D.set_title('Pattern at $y=-2$')
        
        self.ax1D.set_xlim([-4, 4])
        self.ax1D.set_ylim([0, 2])
        self.ax2D.set_ylim([-4, -2])
        self.text = plt.text(2.5, -2.5, "") 
        
        cbar = self.fig.colorbar(self.quadPsi_x, ticks=[-1, 0, 1])
        return
        
    def AnimationStep(self, i):        
        self.linePsi_x.set_ydata(abs((self.Psi_x[i%self.T,int(0.25*self.X)]))**2)
        self.quadPsi_x.set_array(np.ravel(abs((self.Psi_x[i%self.T,:,:]))**2))
        self.text.set_text("t = %.2f" % self.t[i%self.T])
        return (self.linePsi_x, self.quadPsi_x, self.text)

In [82]:
MyTDSE = TDSE_2D()
MyTDSE.SetSpace(xmin=-4, xmax=4, Xsteps=2**7, ymin=-4, ymax=4, Ysteps=2**7)

In [83]:
MyTDSE.SetPotential('Single Slit')
MyTDSE.SimulateSplitOperator(dt=0.01, Tsteps=1000)
MyTDSE.AnimateSetup(interval=10)
ani = animation.FuncAnimation(MyTDSE.fig, MyTDSE.AnimationStep, interval=10, blit=True)
plt.show()

The results (SO), including the diffraction pattern for a single slit experiment:

<img src="Results/Single Slit.png">
    
This diffraction is as can be expected by theory (qualitively).

In [84]:
MyTDSE.SetPotential('Double Slit')
MyTDSE.SimulateSplitOperator(dt=0.01, Tsteps=1000)
MyTDSE.AnimateSetup(interval=10)
ani = animation.FuncAnimation(MyTDSE.fig, MyTDSE.AnimationStep, interval=10, blit=True)
plt.show()

And for the double slit:

<img src="Results/Double Slit.png">

This diffraction is as can be expected by theory (qualitively).